# 1. Data Colection

### Import necessary Library

In [9]:
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import defaultdict
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc

### Load Dataset

In [10]:
df = pd.read_parquet("../dataset/NF_BOT_IoT_V2/NF-BoT-IoT-V2.parquet", engine="pyarrow")  # or engine="fastparquet"
# View the first few rows
df.head()

,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label,Attack
0,-2017,80,6,7.0,140,1,0,0,2,2,...,512,0,0,0,0,0,0,0.0,1,DoS
1,-4762,80,6,7.0,280,2,0,0,2,2,...,512,0,0,0,0,0,0,0.0,1,DoS
2,32534,80,6,7.0,280,2,0,0,2,2,...,512,0,0,0,0,0,0,0.0,1,DoS
3,-13423,80,17,188.0,56,2,0,0,0,0,...,0,0,0,0,0,0,0,0.0,1,DDoS
4,-28756,80,17,188.0,56,2,0,0,0,0,...,0,0,0,0,0,0,0,0.0,1,DDoS


# 2. Explanatory Data Analysis

In [11]:
df.describe()

c:\Users\gech\anaconda3\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: overflow encountered in square
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\gech\anaconda3\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: overflow encountered in square
  sqr = _ensure_numeric((avg - values) ** 2)


,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
count,3.042009e+07,3.042009e+07,3.042009e+07,3.042009e+07,3.042009e+07,3.042009e+07,3.042009e+07,3.042009e+07,3.042009e+07,3.042009e+07,...,3.042009e+07,3.042009e+07,3.042009e+07,3.042009e+07,3.042009e+07,3.042009e+07,3.042009e+07,3.042009e+07,3.042009e+07,3.042009e+07
mean,-9.345863e+02,1.081121e+02,1.116985e+01,9.175757e+01,6.441983e+02,2.840693e+00,2.521820e+02,3.510150e-01,3.665827e+00,1.332763e+00,...,2.744911e-01,4.787784e+02,9.461602e+02,5.286983e+03,2.065228e+01,7.279295e+00,3.086579e-03,6.950239e+00,5.693245e-03,9.957450e-01
std,1.884213e+04,3.835130e+03,5.490616e+00,9.266840e+01,1.141003e+05,1.084273e+02,1.092240e+05,6.997525e+01,7.234989e+00,2.416236e+00,...,6.956851e+01,2.325950e+03,5.147257e+03,1.266814e+04,4.948491e+01,5.639778e+02,2.649712e-01,1.391326e+03,1.591469e+00,6.509132e-02
min,-3.276800e+04,-3.276800e+04,1.000000e+00,0.000000e+00,4.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-5.221800e+04,0.000000e+00,0.000000e+00
25%,-1.723100e+04,8.000000e+01,6.000000e+00,7.000000e+00,5.600000e+01,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
50%,-1.710000e+03,8.000000e+01,6.000000e+00,7.000000e+00,1.120000e+02,2.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,2.000000e+00,...,0.000000e+00,5.120000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,1.541100e+04,8.000000e+01,1.700000e+01,1.880000e+02,2.800000e+02,3.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,2.000000e+00,...,0.000000e+00,5.120000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
max,3.276700e+04,3.276700e+04,5.800000e+01,2.480000e+02,1.890273e+08,4.915300e+04,1.969787e+08,3.786400e+04,2.140000e+02,1.980000e+02,...,3.583800e+04,6.553500e+04,4.240800e+04,6.528000e+04,2.550000e+02,6.553400e+04,2.550000e+02,5.184000e+05,5.300000e+02,1.000000e+00


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30420086 entries, 0 to 30420085
Data columns (total 43 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   L4_SRC_PORT                  int16  
 1   L4_DST_PORT                  int16  
 2   PROTOCOL                     int8   
 3   L7_PROTO                     float32
 4   IN_BYTES                     int32  
 5   IN_PKTS                      int32  
 6   OUT_BYTES                    int32  
 7   OUT_PKTS                     int32  
 8   TCP_FLAGS                    int16  
 9   CLIENT_TCP_FLAGS             int16  
 10  SERVER_TCP_FLAGS             int16  
 11  FLOW_DURATION_MILLISECONDS   int32  
 12  DURATION_IN                  int16  
 13  DURATION_OUT                 int16  
 14  MIN_TTL                      int16  
 15  MAX_TTL                      int16  
 16  LONGEST_FLOW_PKT             int32  
 17  SHORTEST_FLOW_PKT            int16  
 18  MIN_IP_PKT_LEN               int16  
 19

In [13]:
df.shape

(30420086, 43)

In [14]:
df.columns

Index(['L4_SRC_PORT', 'L4_DST_PORT', 'PROTOCOL', 'L7_PROTO', 'IN_BYTES',
       'IN_PKTS', 'OUT_BYTES', 'OUT_PKTS', 'TCP_FLAGS', 'CLIENT_TCP_FLAGS',
       'SERVER_TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS', 'DURATION_IN',
       'DURATION_OUT', 'MIN_TTL', 'MAX_TTL', 'LONGEST_FLOW_PKT',
       'SHORTEST_FLOW_PKT', 'MIN_IP_PKT_LEN', 'MAX_IP_PKT_LEN',
       'SRC_TO_DST_SECOND_BYTES', 'DST_TO_SRC_SECOND_BYTES',
       'RETRANSMITTED_IN_BYTES', 'RETRANSMITTED_IN_PKTS',
       'RETRANSMITTED_OUT_BYTES', 'RETRANSMITTED_OUT_PKTS',
       'SRC_TO_DST_AVG_THROUGHPUT', 'DST_TO_SRC_AVG_THROUGHPUT',
       'NUM_PKTS_UP_TO_128_BYTES', 'NUM_PKTS_128_TO_256_BYTES',
       'NUM_PKTS_256_TO_512_BYTES', 'NUM_PKTS_512_TO_1024_BYTES',
       'NUM_PKTS_1024_TO_1514_BYTES', 'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT',
       'ICMP_TYPE', 'ICMP_IPV4_TYPE', 'DNS_QUERY_ID', 'DNS_QUERY_TYPE',
       'DNS_TTL_ANSWER', 'FTP_COMMAND_RET_CODE', 'Label', 'Attack'],
      dtype='object')

In [15]:
# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [16]:
df = df.sample(frac=0.01, random_state=42)
print(f"Sampled dataset shape: {df.shape}")

Sampled dataset shape: (304201, 43)


# 3. Data Preprocessing

### Handling Missing Values

In [17]:
# Handle missing values: fill numerical with mean, others with 0
for col in df.columns:
    if df[col].dtype in ['int64', 'float64']:
        df[col] = df[col].fillna(df[col].mean())
    else:
        df[col] = df[col].fillna(0)
df.isna().sum()

L4_SRC_PORT                    0
L4_DST_PORT                    0
PROTOCOL                       0
L7_PROTO                       0
IN_BYTES                       0
IN_PKTS                        0
OUT_BYTES                      0
OUT_PKTS                       0
TCP_FLAGS                      0
CLIENT_TCP_FLAGS               0
SERVER_TCP_FLAGS               0
FLOW_DURATION_MILLISECONDS     0
DURATION_IN                    0
DURATION_OUT                   0
MIN_TTL                        0
MAX_TTL                        0
LONGEST_FLOW_PKT               0
SHORTEST_FLOW_PKT              0
MIN_IP_PKT_LEN                 0
MAX_IP_PKT_LEN                 0
SRC_TO_DST_SECOND_BYTES        0
DST_TO_SRC_SECOND_BYTES        0
RETRANSMITTED_IN_BYTES         0
RETRANSMITTED_IN_PKTS          0
RETRANSMITTED_OUT_BYTES        0
RETRANSMITTED_OUT_PKTS         0
SRC_TO_DST_AVG_THROUGHPUT      0
DST_TO_SRC_AVG_THROUGHPUT      0
NUM_PKTS_UP_TO_128_BYTES       0
NUM_PKTS_128_TO_256_BYTES      0
NUM_PKTS_2

### Data Encoding and Transformation

In [18]:
# Encode categorical features (PROTOCOL, L7_PROTO)
le_protocol = LabelEncoder()
df['PROTOCOL'] = le_protocol.fit_transform(df['PROTOCOL'].astype(str))
le_l7_proto = LabelEncoder()
df['L7_PROTO'] = le_l7_proto.fit_transform(df['L7_PROTO'].astype(str))

### Binary Classfication

In [19]:
# Extract labels for binary classification (Label: 0=benign, 1=attack)
y = df['Label'].values

In [20]:
# Select numerical features (exclude Label, Attack)
feature_cols = [col for col in df.columns if col not in ['Label', 'Attack']]
X = df[feature_cols].values

# Normalize numerical features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [21]:
# Create preprocessed DataFrame
df_processed = pd.DataFrame(X, columns=feature_cols)
df_processed['Label'] = y
df_processed['L4_SRC_PORT'] = df['L4_SRC_PORT'].values  # Retain for graph
df_processed['L4_DST_PORT'] = df['L4_DST_PORT'].values
print(f"Preprocessed dataset shape: {df_processed.shape}")

Preprocessed dataset shape: (304201, 42)


In [22]:
# === Graph Construction ===
# Map unique ports to node indices
src_ports = df_processed['L4_SRC_PORT'].values
dst_ports = df_processed['L4_DST_PORT'].values
unique_ports = np.unique(np.concatenate([src_ports, dst_ports]))
port_to_idx = {port: idx for idx, port in enumerate(unique_ports)}

# Aggregate node features and labels
node_features = defaultdict(list)
node_labels = defaultdict(list)
for idx in range(len(df_processed)):
    row = df_processed.iloc[idx]
    src_port = row['L4_SRC_PORT']
    dst_port = row['L4_DST_PORT']
    features = row.drop(['Label', 'L4_SRC_PORT', 'L4_DST_PORT']).values
    label = row['Label']
    node_features[port_to_idx[src_port]].append(features[:-1])  # Exclude label
    node_features[port_to_idx[dst_port]].append(features[:-1])
    node_labels[port_to_idx[src_port]].append(label)
    node_labels[port_to_idx[dst_port]].append(label)

# Compute average features and labels per node
x = []
y = []
for port_idx in range(len(unique_ports)):
    if port_idx in node_features:
        x.append(np.mean(node_features[port_idx], axis=0))
        y.append(np.mean(node_labels[port_idx]))  # Majority for binary
    else:
        x.append(np.zeros(df_processed.shape[1] - 3))  # Default features
        y.append(0)  # Default label

x = torch.tensor(x, dtype=torch.float)
y = torch.tensor(y, dtype=torch.long)

# Construct edges (bidirectional)
edge_index = []
for idx in range(len(df_processed)):
    row = df_processed.iloc[idx]
    src_idx = port_to_idx[row['L4_SRC_PORT']]
    dst_idx = port_to_idx[row['L4_DST_PORT']]
    edge_index.append([src_idx, dst_idx])
    edge_index.append([dst_idx, src_idx])  # Bidirectional

edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

# Create PyTorch Geometric Data object
data = Data(x=x, edge_index=edge_index, y=y)
print(f"Graph constructed: {data.num_nodes} nodes, {data.num_edges} edges")

C:\Users\gech\AppData\Local\Temp\ipykernel_15848\3125400933.py:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  x = torch.tensor(x, dtype=torch.float)


Graph constructed: 64884 nodes, 608402 edges


In [25]:
# === GNN Model Definition ===
# Define GraphSAGE model as a class (required by PyTorch)
class GNNNIDS(torch.nn.Module):
    def __init__(self):
        super(GNNNIDS, self).__init__()
        self.conv1 = SAGEConv(data.num_features, 64)
        self.conv2 = SAGEConv(64, 64)
        self.fc = torch.nn.Linear(64, 2)  # Binary classification

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

# Initialize model and optimizer
model = GNNNIDS()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# === Model Training ===
# Create train/test masks (80% train, 20% test)
num_nodes = data.num_nodes
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)
indices = np.random.permutation(num_nodes)
train_size = int(0.8 * num_nodes)
train_mask[indices[:train_size]] = True
test_mask[indices[train_size:]] = True
data.train_mask = train_mask
data.test_mask = test_mask

# Train model for 100 epochs
model.train()
train_losses = []
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())
    # if (epoch + 1) % 100 == 0:
    print(f'Epoch {epoch + 1}, Loss: {loss.item():.4f}')

# === Step 5: Model Evaluation ===
# Evaluate model 
model.eval()
with torch.no_grad():
    out = model(data)
    pred = out.argmax(dim=1)
    y_true = data.y[data.test_mask].numpy()
    y_pred = pred[data.test_mask].numpy()

    # Compute metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    f1 = f1_score(y_true, y_pred, average='binary')

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig('confusion_matrix.png')
    plt.close()

    # ROC curve
    y_score = out[data.test_mask][:, 1].numpy()
    fpr, tpr, _ = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc='lower right')
    plt.savefig('roc_curve.png')
    plt.close()

# Print evaluation metrics
print("\nEvaluation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1: {f1:.4f}")

# Plot training loss
plt.figure()
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.legend()
plt.savefig('training_loss.png')
plt.close()

Epoch 1, Loss: 0.6244
Epoch 2, Loss: 0.4585
Epoch 3, Loss: 0.3627
Epoch 4, Loss: 0.2231
Epoch 5, Loss: 0.1738
Epoch 6, Loss: 0.1187
Epoch 7, Loss: 0.1021
Epoch 8, Loss: 0.1089
Epoch 9, Loss: 0.1006
Epoch 10, Loss: 0.0864
Epoch 11, Loss: 0.0904
Epoch 12, Loss: 0.0824
Epoch 13, Loss: 0.0804
Epoch 14, Loss: 0.0768
Epoch 15, Loss: 0.0766
Epoch 16, Loss: 0.0832
Epoch 17, Loss: 0.0676
Epoch 18, Loss: 0.0682
Epoch 19, Loss: 0.0642
Epoch 20, Loss: 0.0607
Epoch 21, Loss: 0.0557
Epoch 22, Loss: 0.0617
Epoch 23, Loss: 0.0556
Epoch 24, Loss: 0.0533
Epoch 25, Loss: 0.0537
Epoch 26, Loss: 0.0510
Epoch 27, Loss: 0.0530
Epoch 28, Loss: 0.0505
Epoch 29, Loss: 0.0558
Epoch 30, Loss: 0.0496
Epoch 31, Loss: 0.0480
Epoch 32, Loss: 0.0482
Epoch 33, Loss: 0.0498
Epoch 34, Loss: 0.0476
Epoch 35, Loss: 0.0471
Epoch 36, Loss: 0.0456
Epoch 37, Loss: 0.0461
Epoch 38, Loss: 0.0439
Epoch 39, Loss: 0.0437
Epoch 40, Loss: 0.0430
Epoch 41, Loss: 0.0434
Epoch 42, Loss: 0.0437
Epoch 43, Loss: 0.0467
Epoch 44, Loss: 0.04

In [15]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import roc_curve, auc
from itertools import cycle
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Step 1: Data Preprocessing
def load_and_preprocess_data(file_path, sample_frac=0.03):
    """
    Load and preprocess the NF-BoT-IoT-v2 dataset for multi-class classification.
    Args:
        file_path (str): Path to the CSV file.
        sample_frac (float): Fraction of data to sample (default: 3%).
    Returns:
        df_processed (pd.DataFrame): Preprocessed dataset.
        scaler (StandardScaler): Fitted scaler for numerical features.
        label_encoder (LabelEncoder): Fitted encoder for Attack labels.
    """
    # Load dataset (sample to reduce memory usage)
    # df = pd.read_csv(file_path)
    df = pd.read_parquet(file_path, engine="pyarrow")  # or engine="fastparquet"
    df = df.sample(frac=sample_frac, random_state=42)
    print(f"Sampled dataset shape: {df.shape}")

    # Handle missing values
    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:
            df[col] = df[col].fillna(df[col].mean())
        else:
            df[col] = df[col].fillna(0)

    # Encode categorical features (e.g., PROTOCOL, L7_PROTO)
    categorical_cols = ['PROTOCOL', 'L7_PROTO']
    for col in categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))

    # Encode labels for multi-class classification (Attack column)
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(df['Attack'])
    print(f"Classes: {label_encoder.classes_}")

    # Select numerical features for node features (exclude Label, Attack)
    feature_cols = [col for col in df.columns if col not in ['Label', 'Attack']]
    X = df[feature_cols].values

    # Normalize numerical features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    df_processed = pd.DataFrame(X, columns=feature_cols)
    df_processed['Attack'] = y
    print(f"Preprocessed dataset shape: {df_processed.shape}")

    return df_processed, scaler, label_encoder

# Step 2: Graph Construction
def construct_graph(df_processed):
    """
    Construct a graph where nodes are source/destination ports, and edges represent flows.
    Args:
        df_processed (pd.DataFrame): Preprocessed dataset.
    Returns:
        data (torch_geometric.data.Data): Graph data object for PyG.
    """
    # Map unique ports to node indices
    src_ports = df_processed['L4_SRC_PORT'].values
    dst_ports = df_processed['L4_DST_PORT'].values
    unique_ports = np.unique(np.concatenate([src_ports, dst_ports]))
    port_to_idx = {port: idx for idx, port in enumerate(unique_ports)}

    # Node features: Aggregate flow features for each port
    node_features = defaultdict(list)
    node_labels = defaultdict(list)
    for idx, row in df_processed.iterrows():
        src_port = row['L4_SRC_PORT']
        dst_port = row['L4_DST_PORT']
        features = row.drop(['Attack', 'L4_SRC_PORT', 'L4_DST_PORT']).values
        label = row['Attack']
        
        node_features[port_to_idx[src_port]].append(features[:-1])  # Exclude label
        node_features[port_to_idx[dst_port]].append(features[:-1])
        node_labels[port_to_idx[src_port]].append(label)
        node_labels[port_to_idx[dst_port]].append(label)

    # Average features and take mode for labels per node
    x = []
    y = []
    for port_idx in range(len(unique_ports)):
        if port_idx in node_features:
            x.append(np.mean(node_features[port_idx], axis=0))
            # For multi-class, take mode of labels
            labels = node_labels[port_idx]
            y.append(np.bincount(labels.astype(int)).argmax())
        else:
            x.append(np.zeros(df_processed.shape[1] - 3))  # Default features
            y.append(0)  # Default label

    x = torch.tensor(x, dtype=torch.float)
    y = torch.tensor(y, dtype=torch.long)

    # Construct edges (bidirectional)
    edge_index = []
    for idx, row in df_processed.iterrows():
        src_idx = port_to_idx[row['L4_SRC_PORT']]
        dst_idx = port_to_idx[row['L4_DST_PORT']]
        edge_index.append([src_idx, dst_idx])
        edge_index.append([dst_idx, src_idx])  # Bidirectional

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    # Create PyG Data object
    data = Data(x=x, edge_index=edge_index, y=y)
    print(f"Graph constructed: {data.num_nodes} nodes, {data.num_edges} edges")

    return data

# Step 3: GNN Model Definition
class GNNNIDS(torch.nn.Module):
    """
    Graph Neural Network model for NIDS using GraphSAGE.
    """
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNNIDS, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.fc = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

# Step 4: Model Training
def train_model(data, model, optimizer, epochs=100):
    """
    Train the GNN model with training and validation metrics.
    Args:
        data (Data): Graph data object.
        model (torch.nn.Module): GNN model.
        optimizer: PyTorch optimizer.
        epochs (int): Number of training epochs.
    Returns:
        metrics (dict): Training and validation loss/accuracy per epoch.
    """
    model.train()
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    # Create train/val/test masks (70% train, 15% val, 15% test)
    num_nodes = data.num_nodes
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)
    indices = np.random.permutation(num_nodes)
    train_size = int(0.7 * num_nodes)
    val_size = int(0.15 * num_nodes)
    train_mask[indices[:train_size]] = True
    val_mask[indices[train_size:train_size + val_size]] = True
    test_mask[indices[train_size + val_size:]] = True
    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask

    for epoch in range(epochs):
        # Training
        model.train()
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        # Training metrics
        pred = out.argmax(dim=1)
        train_acc = accuracy_score(data.y[data.train_mask].numpy(), pred[data.train_mask].numpy())
        train_losses.append(loss.item())
        train_accuracies.append(train_acc)

        # Validation
        model.eval()
        with torch.no_grad():
            out = model(data)
            val_loss = F.nll_loss(out[data.val_mask], data.y[data.val_mask]).item()
            val_pred = out.argmax(dim=1)
            val_acc = accuracy_score(data.y[data.val_mask].numpy(), val_pred[data.val_mask].numpy())
            val_losses.append(val_loss)
            val_accuracies.append(val_acc)

        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch + 1}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}, '
                  f'Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')

    return {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accuracies': train_accuracies,
        'val_accuracies': val_accuracies
    }

# Step 5: Model Evaluation
def evaluate_model(data, model, num_classes, label_encoder):
    """
    Evaluate the GNN model for multi-class classification.
    Args:
        data (Data): Graph data object.
        model (torch.nn.Module): Trained GNN model.
        num_classes (int): Number of classes.
        label_encoder (LabelEncoder): Encoder for class labels.
    Returns:
        metrics (dict): Evaluation metrics.
    """
    model.eval()
    with torch.no_grad():
        out = model(data)
        pred = out.argmax(dim=1)
        y_true = data.y[data.test_mask].numpy()
        y_pred = pred[data.test_mask].numpy()

        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred, average='macro', zero_division=0),
            'recall': recall_score(y_true, y_pred, average='macro', zero_division=0),
            'f1': f1_score(y_true, y_pred, average='macro', zero_division=0)
        }

        # Confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.savefig('confusion_matrix.png')
        plt.close()

        # ROC Curve (one-vs-rest for multi-class)
        y_score = F.softmax(out[data.test_mask], dim=1).numpy()
        y_true_bin = np.zeros((y_true.size, num_classes))
        y_true_bin[np.arange(y_true.size), y_true] = 1

        plt.figure(figsize=(8, 6))
        colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red'])
        for i, color in zip(range(num_classes), colors):
            fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_score[:, i])
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, color=color, label=f'{label_encoder.classes_[i]} (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curves (One-vs-Rest)')
        plt.legend(loc='lower right')
        plt.savefig('roc_curve.png')
        plt.close()

    return metrics

# Step 6: Plot Training vs Validation Metrics
def plot_metrics(metrics):
    """
    Plot training vs validation loss and accuracy.
    Args:
        metrics (dict): Training and validation metrics.
    """
    epochs = range(1, len(metrics['train_losses']) + 1)

    # Loss plot
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, metrics['train_losses'], label='Training Loss')
    plt.plot(epochs, metrics['val_losses'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training vs Validation Loss')
    plt.legend()
    plt.savefig('train_val_loss.png')
    plt.close()

    # Accuracy plot
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, metrics['train_accuracies'], label='Training Accuracy')
    plt.plot(epochs, metrics['val_accuracies'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training vs Validation Accuracy')
    plt.legend()
    plt.savefig('train_val_accuracy.png')
    plt.close()

# Main Execution
def main():
    # File path to dataset
    file_path = 'NF-BoT-IoT-V2.parquet'  # Replace with actual path

    # Step 1: Preprocess data
    df_processed, scaler, label_encoder = load_and_preprocess_data(file_path)

    # Step 2: Construct graph
    data = construct_graph(df_processed)

    # Step 3: Initialize model
    num_classes = len(label_encoder.classes_)
    model = GNNNIDS(
        in_channels=data.num_features,
        hidden_channels=64,
        out_channels=num_classes
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    # Step 4: Train model
    metrics = train_model(data, model, optimizer)

    # Step 5: Evaluate model
    eval_metrics = evaluate_model(data, model, num_classes, label_encoder)
    print("\nEvaluation Metrics:")
    for key, value in eval_metrics.items():
        print(f"{key.capitalize()}: {value:.4f}")

    # Step 6: Plot training vs validation metrics
    plot_metrics(metrics)

if __name__ == "__main__":
    main()

ArrowMemoryError: malloc of size 121680384 failed